In [ ]:
import os
import logging
import pandas as pd
logger = logging.getLogger('LabJournal')
logging.basicConfig(level=logging.DEBUG)

In [ ]:
import sys
sys.path.append("../") # need this if labjournal is not installed

from simdb.utils.fileFinder import find_files
from simdb.databaseModel import *

In [ ]:
SIMS = '/home/andrejb/Research/SIMS'

In [ ]:
logger.info('create_database:create_database: FIND FILES')
ERRORS=False
WARNINGS=False

In [ ]:
%%time
SIM_IDS=[]
PATHS=[]
METAS=[]

for fname in find_files(pattern = 'topol.tpr', path = SIMS, dir_ignore = ['data']):
    path =  os.path.dirname(fname)
    sim_id = os.path.basename(path)
    meta = pd.Series.from_csv(os.path.join(path, "meta.csv"))
    SIM_IDS.append(sim_id)
    PATHS.append(path)
    METAS.append(meta)

In [ ]:
db = 'andrej_raw.db'
# remove old DB
try:
    os.remove(db)
    logger.info('create_database:create_database: removed old file: %s', db)
except:
    pass
engine = create_engine('sqlite:///{}'.format(db) , echo=False) #  if we want spam

# Establishing a session
Session = sessionmaker(bind=engine)
session = Session()
setup_database(engine)

In [ ]:
%% time
for i, entry_id in enumerate(SIM_IDS):
    meta = METAS[i]
    try:
        description = meta['note']
    except KeyError:
        description = ""
    sim = Main(
        entry_id    = entry_id,
        path        = PATHS[i],
        sim_type    = "GROMACS",
        description = description
    )
    session.add(sim)
    session.flush()
    
    for k, v in meta.iteritems():
        if k != 'note':
            key = Keywords(
                main_id = sim.id,
                name = k,
                value = v
            )
            session.add(key)
session.commit()
session.close()

In [ ]:
SIM_IDS